# Set up

### Import Python packages to use in the script

In [20]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
import skan
from imaris_ims_file_reader.ims import ims
from tqdm import tqdm

### Define the functions that will run different steps/tasks on the data

Need to select which GPU device to send data to

In [11]:
cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              573.24
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

In [12]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [13]:
def get_measurements(mask,img,props):
    filtered_masks = sk.segmentation.clear_border(mask)
    size_filtered = sk.morphology.remove_small_objects(filtered_masks,min_size=2000)
    df = sk.measure.regionprops_table(size_filtered,img,properties=props)
    df = pd.DataFrame.from_dict(df)
    return size_filtered, df

In [14]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [15]:
def get_3D_surfacearea_and_sphericity(mask_img,scaled_df):
    surf_area_list = []
    sphericity_list = []
    vol = np.asarray(scaled_df['Volume (um^3)']).astype(np.float64)
    c = 0
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=(voxel_size_z,voxel_size_x,voxel_size_y))
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphericity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphericity_list.append(sphericity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphericities = pd.Series(sphericity_list,name='Sphericity')
    merged_df = pd.concat([scaled_df,surface_areas,sphericities], axis=1)
    return merged_df

Below cell contains the formula for calculating sphericity and the source of the code. Not required to run below cell for script to work.

In [ ]:
# def sphericity(mesh_volume, mesh_surface_area):
    
#     '''This definition of sphericity assumes that you are working in continuous space.
#     Parameters:
#     -----------
#     mesh_volume: integer or float value
#     mesh_surface_area: integer or float value
#     Returns:
#     --------
#     psi: a float value with range(0,1) reflecting the compactness of an object
#     Got this script from: https://github.com/BiAPoL/Bio-image_Analysis_with_Python/blob/main/05_feature_extraction/02_sphericity_and_solidity.ipynb
#     '''
#     numerator = (np.pi ** (1/3)) * ((6 * mesh_volume) ** (2/3))
#     denominator = mesh_surface_area
    
#     psi = numerator / denominator
    
#     return psi

In [16]:
def save(save_path, img_name, norm_img, filtered_masks, merged_df):
    # make directories if they do not exist
    try:
        norm_path = os.mkdir(os.path.join(save_path,'norm_imgs'))
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        norm_path = os.path.join(save_path,'norm_imgs')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    imsave(os.path.join(norm_path,'normalized_'+img_name[:-4]+'.tif'),norm_img)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Load in Data

## Two options for loading in data:
Note that options 1 and 2 expect the images to be in a tiff format. sk.io.imread cannot read in ims files

### Option 1: Specify the locations of all image containing folders:
Then merge into a long list of images and file names

In [ ]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [ ]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [ ]:
all_files = CMO_files+GMO_1ng_files+GMO_2ng_files # need this for getting image names later
all_imgs = CMO_imgs+GMO_1ng_imgs+GMO_2ng_imgs

### Option 2: Use a recursive search option to find all images in the parent directory:

In [21]:
all_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/**/*.tif',recursive=True)) #finds all tiff images in the subdirectories of TIF_Files
all_imgs = list(map(sk.io.imread,all_files))

### Option 3: Read in Imaris files directly

In [22]:
all_files = sorted(glob('./**/*.ims'))

In [23]:
all_imgs = [ims(img,ResolutionLevelLock=0) for img in all_files]

Opening readonly file: .\ImarisFiles\120 uM.ims 

Opening readonly file: .\ImarisFiles\120 uM_1.ims 

Opening readonly file: .\ImarisFiles\120 uM_10.ims 

Opening readonly file: .\ImarisFiles\120 uM_11.ims 

Opening readonly file: .\ImarisFiles\120 uM_12.ims 

Opening readonly file: .\ImarisFiles\120 uM_13.ims 

Opening readonly file: .\ImarisFiles\120 uM_14.ims 

Opening readonly file: .\ImarisFiles\120 uM_15.ims 

Opening readonly file: .\ImarisFiles\120 uM_16.ims 

Opening readonly file: .\ImarisFiles\120 uM_2.ims 

Opening readonly file: .\ImarisFiles\120 uM_3.ims 

Opening readonly file: .\ImarisFiles\120 uM_4.ims 

Opening readonly file: .\ImarisFiles\120 uM_5.ims 

Opening readonly file: .\ImarisFiles\120 uM_6.ims 

Opening readonly file: .\ImarisFiles\120 uM_7.ims 

Opening readonly file: .\ImarisFiles\120 uM_8.ims 

Opening readonly file: .\ImarisFiles\120 uM_9.ims 

Opening readonly file: .\ImarisFiles\40 uM.ims 

Opening readonly file: .\ImarisFiles\40 uM_1.ims 

Opening rea

In [24]:
all_imgs = [img[0,0,:,:,:] for img in all_imgs]

Closing file: .\ImarisFiles\control_9.ims 

Closing file: .\ImarisFiles\control_8.ims 

Closing file: .\ImarisFiles\control_7.ims 

Closing file: .\ImarisFiles\control_6.ims 

Closing file: .\ImarisFiles\control_5.ims 

Closing file: .\ImarisFiles\control_4.ims 

Closing file: .\ImarisFiles\control_3.ims 

Closing file: .\ImarisFiles\control_2.ims 

Closing file: .\ImarisFiles\control_15.ims 

Closing file: .\ImarisFiles\control_14.ims 

Closing file: .\ImarisFiles\control_13.ims 

Closing file: .\ImarisFiles\control_12.ims 

Closing file: .\ImarisFiles\control_11.ims 

Closing file: .\ImarisFiles\control_10.ims 

Closing file: .\ImarisFiles\control_1.ims 

Closing file: .\ImarisFiles\control.ims 

Closing file: .\ImarisFiles\80 uM_9.ims 

Closing file: .\ImarisFiles\80 uM_8.ims 

Closing file: .\ImarisFiles\80 uM_7.ims 

Closing file: .\ImarisFiles\80 uM_6.ims 

Closing file: .\ImarisFiles\80 uM_5.ims 

Closing file: .\ImarisFiles\80 uM_4.ims 

Closing file: .\ImarisFiles\80 uM_3.ims 

In [10]:
all_imgs[0].shape

(41, 2048, 2048)

# Run Pipeline

### Optional use of random integer generator if you want to test on single random images first

In [ ]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))

In [ ]:
tophat_radius = 10
img = normalize_images(all_img[i],tophat_radius) #change the tophat_radius to increase or decrease the background as needed

In [ ]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(img, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

In [ ]:
#View results of segmentation in Napari
viewer = napari.view_image(all_img[i], name='original img', scale = (4.55,0.3,0.3))
viewer.add_image(img,name='normalized img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

### Run pipeline on all images

Set up a few parameters before running

Properties that can be measured with scikit-image can be found here: [Scikit-image Region Properties](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops)

In [30]:
tophat_radius = 10
voxel_size_x = 0.301
voxel_size_y = 0.301
voxel_size_z = 4.55
voxel = 0.301*0.301*4.55
save_path = 'E:/ParadaKusz_Lab/OIC-154_Microglia_Quantification/Segmentation_Outputs'
props = ('label','area','intensity_mean','intensity_min','intensity_max')

In [26]:
norm_imgs = [normalize_images(img,tophat_radius) for img in tqdm(all_imgs)]

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [12:07<00:00, 11.20s/it]


In [27]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(norm_imgs, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

2025-06-03 15:46:21,878 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-03 15:46:21,880 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-03 15:46:22,024 [INFO] ** TORCH CUDA version installed and working. **
2025-06-03 15:46:22,025 [INFO] >>>> using GPU (CUDA)
2025-06-03 15:46:22,028 [INFO] >> cyto3 << model set to be used
2025-06-03 15:46:22,111 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-06-03 15:46:22,274 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-03 15:46:22,277 [INFO] channels set to [[0, 0]]
2025-06-03 15:46:22,278 [INFO] ~~~ FINDING MASKS ~~~
2025-06-03 15:46:22,279 [INFO] 0%|          | 0/65 [00:00<?, ?it/s]
2025-06-03 15:46:22,280 [INFO] multi-stack tiff read in as having 41 planes 1 channels
2025-06-03 15:46:24,622 [INFO] 
2025-06-03 15:46:24,623 [INFO] 0%|          | 0/41 [00:00<?, ?it/s]
20


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 12.77it/s]


2025-06-03 15:47:44,999 [INFO] masks created in 69.36s
2025-06-03 15:47:59,510 [INFO] 2%|1         | 1/65 [01:37<1:43:42, 97.23s/it]
2025-06-03 15:47:59,512 [INFO] multi-stack tiff read in as having 48 planes 1 channels
2025-06-03 15:48:01,994 [INFO] 
2025-06-03 15:48:01,997 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 15:48:01,999 [INFO] 
2025-06-03 15:48:12,557 [INFO] 100%|##########| 48/48 [00:10<00:00,  4.55it/s]
2025-06-03 15:48:14,164 [INFO] network run in 12.17s
2025-06-03 15:48:14,166 [INFO] 
2025-06-03 15:48:14,167 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 15:48:14,168 [INFO] 
2025-06-03 15:48:44,610 [INFO] 
2025-06-03 15:48:44,612 [INFO] 48%|####7     | 23/48 [00:30<00:33,  1.32s/it]
2025-06-03 15:48:44,614 [INFO] 
2025-06-03 15:48:56,689 [INFO] 
2025-06-03 15:48:56,690 [INFO] 48%|####7     | 23/48 [00:42<00:33,  1.32s/it]
2025-06-03 15:48:56,692 [INFO] 
2025-06-03 15:49:14,228 [INFO] 100%|##########| 48/48 [01:00<00:00,  1.25s/it]
2025-06-03 15:49:14,2


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:02<00:00, 23.36it/s]


2025-06-03 15:49:20,065 [INFO] masks created in 65.90s
2025-06-03 15:49:39,376 [INFO] 3%|3         | 2/65 [03:17<1:43:43, 98.78s/it]
2025-06-03 15:49:39,378 [INFO] multi-stack tiff read in as having 44 planes 1 channels
2025-06-03 15:49:41,844 [INFO] 
2025-06-03 15:49:41,846 [INFO] 0%|          | 0/44 [00:00<?, ?it/s]
2025-06-03 15:49:41,847 [INFO] 
2025-06-03 15:49:51,702 [INFO] 100%|##########| 44/44 [00:09<00:00,  4.47it/s]
2025-06-03 15:49:53,284 [INFO] network run in 11.44s
2025-06-03 15:49:53,286 [INFO] 
2025-06-03 15:49:53,287 [INFO] 0%|          | 0/44 [00:00<?, ?it/s]
2025-06-03 15:49:53,289 [INFO] 
2025-06-03 15:50:23,388 [INFO] 
2025-06-03 15:50:23,390 [INFO] 48%|####7     | 21/44 [00:30<00:32,  1.43s/it]
2025-06-03 15:50:23,392 [INFO] 
2025-06-03 15:50:36,994 [INFO] 
2025-06-03 15:50:36,995 [INFO] 48%|####7     | 21/44 [00:43<00:32,  1.43s/it]
2025-06-03 15:50:36,997 [INFO] 
2025-06-03 15:50:53,516 [INFO] 
2025-06-03 15:50:53,518 [INFO] 100%|##########| 44/44 [01:00<00:00, 

...

2025-06-03 17:20:30,017 [INFO] masks created in 52.72s
2025-06-03 17:20:47,415 [INFO] 97%|#########6| 63/65 [1:34:25<02:45, 82.83s/it]
2025-06-03 17:20:47,416 [INFO] multi-stack tiff read in as having 48 planes 1 channels
2025-06-03 17:20:50,015 [INFO] 
2025-06-03 17:20:50,017 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 17:20:50,017 [INFO] 
2025-06-03 17:21:00,092 [INFO] 100%|##########| 48/48 [00:10<00:00,  4.77it/s]
2025-06-03 17:21:01,651 [INFO] network run in 11.64s
2025-06-03 17:21:01,653 [INFO] 
2025-06-03 17:21:01,654 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 17:21:01,655 [INFO] 
2025-06-03 17:21:32,605 [INFO] 
2025-06-03 17:21:32,607 [INFO] 56%|#####6    | 27/48 [00:30<00:24,  1.15s/it]
2025-06-03 17:21:32,609 [INFO] 
2025-06-03 17:21:44,142 [INFO] 
2025-06-03 17:21:44,143 [INFO] 56%|#####6    | 27/48 [00:42<00:24,  1.15s/it]
2025-06-03 17:21:44,145 [INFO] 
2025-06-03 17:21:56,758 [INFO] 100%|##########| 48/48 [00:55<00:00,  1.15s/it]
2025-06-03 17:21:56


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:02<00:00, 23.37it/s]


2025-06-03 17:22:02,302 [INFO] masks created in 60.65s
2025-06-03 17:22:19,993 [INFO] 98%|#########8| 64/65 [1:35:57<01:25, 85.76s/it]
2025-06-03 17:22:19,994 [INFO] multi-stack tiff read in as having 48 planes 1 channels
2025-06-03 17:22:23,011 [INFO] 
2025-06-03 17:22:23,014 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 17:22:23,016 [INFO] 
2025-06-03 17:22:33,403 [INFO] 100%|##########| 48/48 [00:10<00:00,  4.62it/s]
2025-06-03 17:22:35,132 [INFO] network run in 12.12s
2025-06-03 17:22:35,134 [INFO] 
2025-06-03 17:22:35,135 [INFO] 0%|          | 0/48 [00:00<?, ?it/s]
2025-06-03 17:22:35,137 [INFO] 
2025-06-03 17:23:05,742 [INFO] 
2025-06-03 17:23:05,744 [INFO] 56%|#####6    | 27/48 [00:30<00:23,  1.13s/it]
2025-06-03 17:23:05,746 [INFO] 
2025-06-03 17:23:24,219 [INFO] 
2025-06-03 17:23:24,221 [INFO] 56%|#####6    | 27/48 [00:49<00:23,  1.13s/it]
2025-06-03 17:23:24,223 [INFO] 
2025-06-03 17:23:25,544 [INFO] 100%|##########| 48/48 [00:50<00:00,  1.05s/it]
2025-06-03 17:23:25


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:01<00:00, 24.47it/s]


2025-06-03 17:23:30,871 [INFO] masks created in 55.74s
2025-06-03 17:23:47,814 [INFO] 100%|##########| 65/65 [1:37:25<00:00, 89.93s/it]
2025-06-03 17:23:47,816 [INFO] >>>> TOTAL TIME 5845.54 sec


In [ ]:
for i in tqdm(range(len(norm_imgs))):
    img_name = os.path.basename(all_files[i])
    mask_img, df = get_measurements(masks[i],all_imgs[i],props)
    scaled_df = scaled_vol(df,voxel)
    merged_df = get_3D_surfacearea_and_sphericity(mask_img,scaled_df)
    save(save_path, img_name, norm_imgs[i], mask_img, merged_df)

  0%|                                                                                           | 0/65 [00:00<?, ?it/s]